In [1]:
%%writefile upload_multiple_runs_to_separate_projects.py
import wandb
import json
import yaml
import os
from kaggle_secrets import UserSecretsClient

BASE_PATH = '/kaggle/input/rt-detr-dinov3-distilled-model/FINAL'

runs_to_upload = [
    {
        'project_name': 'rtdetr-taco-convnext-teacher',
        'output_dir': os.path.join(BASE_PATH, 'FINETUNE_DISTILLED/rtdetrv2_finetune_taco_convnext_teacher'),
        'config_file_path': os.path.join(BASE_PATH, 'CONFIG/rtdetrv2_taco_finetune_convnext.yml'),
        'run_name': 'finetune_convnext_teacher',
        'artifact_name': 'rtdetrv2-convnext-teacher-best'
    },
    {
        'project_name': 'rtdetr-taco-vit-teacher',
        'output_dir': os.path.join(BASE_PATH, 'FINETUNE_DISTILLED/rtdetrv2_finetune_taco_vit_teacher'),
        'config_file_path': os.path.join(BASE_PATH, 'CONFIG/rtdetrv2_taco_finetune_vit.yml'),
        'run_name': 'finetune_vit_teacher',
        'artifact_name': 'rtdetrv2-vit-teacher-best'
    },
    {
        'project_name': 'rtdetr-taco-baseline',
        'output_dir': os.path.join(BASE_PATH, 'FINETUNE_BASELINE/rtdetrv2_finetune_taco_BASELINE'),
        'config_file_path': os.path.join(BASE_PATH, 'CONFIG/rtdetrv2_taco_finetune_BASELINE.yml'),
        'run_name': 'finetune_baseline',
        'artifact_name': 'rtdetrv2-baseline-best'
    },
]

def upload_single_run(project_name, output_dir, config_file_path, run_name, artifact_name):
    log_file_path = os.path.join(output_dir, 'log.txt')
    checkpoint_file_path = os.path.join(output_dir, 'best.pth')

    print(f"\n--- Starting upload for project: {project_name}, run: {run_name} ---")

    if not os.path.exists(log_file_path):
        print(f"ERROR: Log file not found, skipping run: {log_file_path}")
        return
    if not os.path.exists(checkpoint_file_path):
        print(f"ERROR: Checkpoint file not found, skipping run: {checkpoint_file_path}")
        return

    run_config = None
    try:
        with open(config_file_path, 'r') as f:
            run_config = yaml.safe_load(f)
        
        if run_config and '__include__' in run_config:
            del run_config['__include__']
            
        print(f"Successfully read and processed config file: {config_file_path}")
    except FileNotFoundError:
        print(f"Warning: Config file not found at '{config_file_path}'. Skipping config logging.")
    except Exception as e:
        print(f"Warning: Could not read config file. Error: {e}")
        
    run = wandb.init(
        project=project_name,
        name=run_name,
        config=run_config,
        job_type='manual-upload',
        reinit=True
    )

    print(f"Parsing and logging metrics from: {log_file_path}")
    with open(log_file_path, 'r') as f:
        for line in f:
            try:
                log_data = json.loads(line)
                epoch = log_data.get('epoch')
                if epoch is not None:
                    metrics_to_log = {k: v for k, v in log_data.items() if k not in ['epoch', 'n_parameters']}
                    wandb.log(metrics_to_log, step=epoch)
            except json.JSONDecodeError:
                print(f"Warning: Could not parse line in log.txt: {line.strip()}")

    print(f"Logging model artifact from: {checkpoint_file_path}")
    artifact = wandb.Artifact(name=artifact_name, type='model')
    artifact.add_file(checkpoint_file_path)
    run.log_artifact(artifact)

    run.finish()
    print(f"--- Finished upload for run: {run_name} ---")


def main():
    try:
        secrets = UserSecretsClient()
        wandb_key = secrets.get_secret("WANDB_API_KEY")
        wandb.login(key=wandb_key)
    except Exception as e:
        print(f"Could not log in to W&B. Please ensure 'WANDB_API_KEY' is set in Kaggle Secrets. Error: {e}")
        return

    for run_info in runs_to_upload:
        upload_single_run(
            project_name=run_info['project_name'],
            output_dir=run_info['output_dir'],
            config_file_path=run_info['config_file_path'],
            run_name=run_info['run_name'],
            artifact_name=run_info['artifact_name']
        )
    
    print("\nAll specified runs have been uploaded.")

if __name__ == '__main__':
    main()

Writing upload_multiple_runs_to_separate_projects.py


In [2]:
!python upload_multiple_runs_to_separate_projects.py

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [3]:
!pip install -q thop ultralytics pycocotools faster-coco-eval protobuf==3.20.3
!pip install -q "numpy<2.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 579.2/579.2 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
%%writefile /kaggle/working/final_benchmark.py
import os
import sys
import subprocess
import torch
import warnings
import yaml
import shutil
import json
import re
import time
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import OrderedDict

RTDETR_REPO_PATH = "/kaggle/working/RT-DETR"
if not os.path.exists(RTDETR_REPO_PATH):
    print(f"ERROR: Repository not found at '{RTDETR_REPO_PATH}'. Please run the git clone cell first.")
    sys.exit(1)

rtdetr_src_path = os.path.join(RTDETR_REPO_PATH, "rtdetrv2_pytorch")
if rtdetr_src_path not in sys.path:
    sys.path.insert(0, rtdetr_src_path)

from ultralytics import YOLO
from thop import profile
from src.core import YAMLConfig

def run_command(command, working_dir=None):
    print(f"\n---> Executing command: {' '.join(command)}")
    full_output = []
    try:
        process = subprocess.Popen(
            command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
            text=True, encoding='utf-8', cwd=working_dir
        )
        for line in process.stdout:
            print(line.strip())
            full_output.append(line.strip())
        process.wait()
        print(f"--- Command {'succeeded' if process.returncode == 0 else f'failed with exit code {process.returncode}'} ---")
        return process.returncode, "\n".join(full_output)
    except Exception as e:
        print(f"An error occurred while executing command: {e}")
        return -1, str(e)

def print_header(title):
    print("\n" + "="*80 + f"\n| {title:^76} |\n" + "="*80)

def merge_yamls(base, new):
    for key, value in new.items():
        if key in base and isinstance(base[key], dict) and isinstance(value, dict): merge_yamls(base[key], value)
        else: base[key] = value
    return base

def flatten_yaml_config(config_path, repo_configs_dir):
    with open(config_path, 'r') as f: config = yaml.safe_load(f) or {}
    if '__include__' in config:
        includes = config.pop('__include__')
        base_config = {}
        for include_path_relative in includes:
            include_filename = os.path.basename(include_path_relative)
            found_path = next((os.path.join(root, include_filename) for root, _, files in os.walk(repo_configs_dir) if include_filename in files), None)
            if found_path:
                included_config = flatten_yaml_config(found_path, repo_configs_dir)
                base_config = merge_yamls(base_config, included_config)
            else:
                raise FileNotFoundError(f"Could not find included file '{include_filename}' within '{repo_configs_dir}'")
        config = merge_yamls(base_config, config)
    return config

def parse_rtdetr_output(output_text):
    metrics = {}
    try:
        ap_50_95_line = re.search(r"Average Precision\s+\(AP\) @\[ IoU=0.50:0.95 .* = ([\d\.]+)", output_text)
        ap_50_line = re.search(r"Average Precision\s+\(AP\) @\[ IoU=0.50\s+.* = ([\d\.]+)", output_text)
        if ap_50_95_line: metrics['mAP50-95'] = float(ap_50_95_line.group(1))
        if ap_50_line: metrics['mAP50'] = float(ap_50_line.group(1))
    except Exception as e:
        print(f"Could not parse RT-DETR metrics: {e}")
    return metrics

def prepare_yolo_dataset():
    print_header("Preparing Dataset for YOLO")
    YOLO_DATA_ROOT = '/kaggle/working/taco_yolo'
    IMAGE_TRAIN_DIR_SRC = '/kaggle/input/dsp-pre-final/processed_taco_coco/train2017'
    IMAGE_VAL_DIR_SRC = '/kaggle/input/dsp-pre-final/processed_taco_coco/val2017'
    COCO_ANNOTATIONS_TRAIN = '/kaggle/input/dsp-pre-final/processed_taco_coco/annotations/instances_train2017.json'
    IMAGE_TRAIN_DIR_DEST = os.path.join(YOLO_DATA_ROOT, 'images', 'train')
    IMAGE_VAL_DIR_DEST = os.path.join(YOLO_DATA_ROOT, 'images', 'val')
    LABEL_TRAIN_DIR_DEST = os.path.join(YOLO_DATA_ROOT, 'labels', 'train')
    LABEL_VAL_DIR_DEST = os.path.join(YOLO_DATA_ROOT, 'labels', 'val')
    for d in [IMAGE_TRAIN_DIR_DEST, IMAGE_VAL_DIR_DEST, LABEL_TRAIN_DIR_DEST, LABEL_VAL_DIR_DEST]: os.makedirs(d, exist_ok=True)
    os.system(f'cp -r {IMAGE_TRAIN_DIR_SRC}/* {IMAGE_TRAIN_DIR_DEST}/ 2>/dev/null')
    os.system(f'cp -r {IMAGE_VAL_DIR_SRC}/* {IMAGE_VAL_DIR_DEST}/ 2>/dev/null')
    def convert_coco_to_yolo(json_file, output_labels_dir):
        with open(json_file) as f: data = json.load(f)
        images_map = {img['id']: (img['file_name'], img['width'], img['height']) for img in data['images']}
        for ann in tqdm(data['annotations'], desc=f"Converting {os.path.basename(json_file)}"):
            image_id, class_id = ann['image_id'], ann['category_id']
            if image_id not in images_map: continue
            file_name, img_w, img_h = images_map[image_id]
            box = ann['bbox']
            x, y, w, h = box; x_center, y_center = (x + w / 2) / img_w, (y + h / 2) / img_h; norm_w, norm_h = w / img_w, h / img_h
            label_file_name = os.path.splitext(file_name)[0] + '.txt'
            with open(os.path.join(output_labels_dir, label_file_name), 'a') as f: f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {norm_w:.6f} {norm_h:.6f}\n")
    convert_coco_to_yolo(COCO_ANNOTATIONS_TRAIN, LABEL_TRAIN_DIR_DEST)
    convert_coco_to_yolo('/kaggle/input/dsp-pre-final/processed_taco_coco/annotations/instances_val2017.json', LABEL_VAL_DIR_DEST)
    with open(COCO_ANNOTATIONS_TRAIN) as f: coco_data = json.load(f)
    categories = sorted(coco_data['categories'], key=lambda x: x['id']); class_names = [cat['name'] for cat in categories]
    taco_yaml_content = {'path': YOLO_DATA_ROOT, 'train': 'images/train', 'val': 'images/val', 'nc': len(class_names), 'names': class_names}
    YAML_PATH = os.path.join(YOLO_DATA_ROOT, 'taco.yaml')
    with open(YAML_PATH, 'w') as f: yaml.dump(taco_yaml_content, f, sort_keys=False)
    print(f"YOLO dataset YAML created at: {YAML_PATH}")
    return YAML_PATH

def setup_and_verify(base_path):
    print_header("Step 1: Verifying Checkpoint and Config Files")
    paths = {
        "Distill ConvNext Finetune": { "type": "rtdetr", "weights": f"{base_path}/FINETUNE_DISTILLED/rtdetrv2_finetune_taco_convnext_teacher/best.pth", "config_src": f"{base_path}/CONFIG/rtdetrv2_taco_finetune_convnext.yml" },
        "Distill ViT Finetune": { "type": "rtdetr", "weights": f"{base_path}/FINETUNE_DISTILLED/rtdetrv2_finetune_taco_vit_teacher/best.pth", "config_src": f"{base_path}/CONFIG/rtdetrv2_taco_finetune_vit.yml" },
        "RT-DETR Baseline Finetune": { "type": "rtdetr", "weights": f"{base_path}/FINETUNE_BASELINE/rtdetrv2_finetune_taco_BASELINE/best.pth", "config_src": f"{base_path}/CONFIG/rtdetrv2_taco_finetune_BASELINE.yml" },
        "YOLOv11 Finetune": { "type": "yolo", "weights": f"{base_path}/YOLO/yolo_checkpoints/yolo11l_finetune_baseline/weights/best.pt"}
    }
    found_paths = {}
    for name, path_dict in paths.items():
        if all(os.path.exists(p) for k, p in path_dict.items() if k != 'type'):
            print(f"[FOUND] {name}"); found_paths[name] = path_dict
        else: print(f"[SKIPPING] {name} - Missing files.")
    if not found_paths: sys.exit(1)
    print("\n---> All required files are ready!")
    return found_paths

def evaluate_models(paths, yolo_data_yaml, YOLO_class, benchmark_results):
    print_header("Step 2: Evaluating Performance (APval / AP50val)")
    rtdetr_work_dir = "/kaggle/working/RT-DETR/rtdetrv2_pytorch"
    repo_configs_dir = os.path.join(rtdetr_work_dir, "configs")
    for name, path_dict in paths.items():
        benchmark_results[name] = {}
        print(f"\n--- Evaluating: {name} ---")
        if path_dict["type"] == "rtdetr":
            config_data = flatten_yaml_config(path_dict["config_src"], repo_configs_dir)
            if 'tuning' in config_data: del config_data['tuning']
            if 'val_dataloader' in config_data:
                config_data['val_dataloader']['dataset']['img_folder'] = "/kaggle/input/dsp-pre-final/processed_taco_coco/val2017"
                config_data['val_dataloader']['dataset']['ann_file'] = "/kaggle/input/dsp-pre-final/processed_taco_coco/annotations/instances_val2017.json"
            temp_config_path = "/kaggle/working/temp_flattened_config.yml"
            with open(temp_config_path, 'w') as f: yaml.dump(config_data, f)
            command = ["python", "tools/train.py", "-c", temp_config_path, "-r", path_dict["weights"], "--test-only"]
            return_code, output = run_command(command, rtdetr_work_dir)
            if return_code == 0: benchmark_results[name].update(parse_rtdetr_output(output))
            if os.path.exists(temp_config_path): os.remove(temp_config_path)
        elif path_dict["type"] == "yolo":
            model = YOLO_class(path_dict["weights"])
            results = model.val(data=yolo_data_yaml, imgsz=640, batch=16, split='val')
            if results:
                benchmark_results[name]['mAP50-95'] = results.box.map
                benchmark_results[name]['mAP50'] = results.box.map50
    
def calculate_complexity(paths, YOLO_class, profile_func, YAMLConfig_class, benchmark_results):
    print_header("Step 3: Analyzing Model Complexity (Params & FLOPs)")
    dummy_input = torch.randn(1, 3, 640, 640)
    rtdetr_work_dir = "/kaggle/working/RT-DETR/rtdetrv2_pytorch"
    repo_configs_dir = os.path.join(rtdetr_work_dir, "configs")
    for name, path_dict in paths.items():
        print(f"\n--- Analyzing: {name} ---")
        try:
            model_cpu = None
            if path_dict["type"] == "rtdetr":
                config_data = flatten_yaml_config(path_dict["config_src"], repo_configs_dir)
                temp_config_path = f"/kaggle/working/temp_complexity_{name.replace(' ', '_')}.yml"
                with open(temp_config_path, 'w') as f: yaml.dump(config_data, f)
                cfg = YAMLConfig(temp_config_path)
                model_cpu = cfg.model.cpu()
                if os.path.exists(temp_config_path): os.remove(temp_config_path)
            elif path_dict["type"] == "yolo":
                model_cpu = YOLO_class(path_dict["weights"]).model.cpu()
            if model_cpu:
                macs, params = profile_func(model_cpu, inputs=(dummy_input.cpu(),), verbose=False)
                benchmark_results[name]['Params (M)'] = params / 1e6
                benchmark_results[name]['FLOPs (G)'] = macs * 2 / 1e9
                print(f"Params: {benchmark_results[name]['Params (M)']:.2f} M, FLOPs: {benchmark_results[name]['FLOPs (G)']:.2f} G")
        except Exception as e:
            print(f"Error analyzing {name}: {e}")

def measure_inference_speed(paths, YOLO_class, YAMLConfig_class, benchmark_results):
    print_header("Step 4: Measuring Inference Speed")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dummy_input = torch.randn(1, 3, 640, 640, device=device)
    warmup_runs = 20
    timed_runs = 50
    rtdetr_work_dir = "/kaggle/working/RT-DETR/rtdetrv2_pytorch"
    repo_configs_dir = os.path.join(rtdetr_work_dir, "configs")

    for name, path_dict in paths.items():
        print(f"\n--- Measuring: {name} ---")
        try:
            model = None
            if path_dict["type"] == "rtdetr":
                config_data = flatten_yaml_config(path_dict["config_src"], repo_configs_dir)
                temp_config_path = f"/kaggle/working/temp_speed_{name.replace(' ', '_')}.yml"
                with open(temp_config_path, 'w') as f: yaml.dump(config_data, f)
                cfg = YAMLConfig(temp_config_path)
                if os.path.exists(temp_config_path): os.remove(temp_config_path)
                
                model = cfg.model
                state_dict = torch.load(path_dict["weights"], map_location=device)
                if 'model' in state_dict:
                    model.load_state_dict(state_dict['model'])
                else:
                    model.load_state_dict(state_dict)
                model.to(device).eval()

            elif path_dict["type"] == "yolo":
                model = YOLO_class(path_dict["weights"]).model.to(device).eval()

            if model:
                with torch.no_grad():
                    for _ in range(warmup_runs): _ = model(dummy_input)
                    torch.cuda.synchronize()
                    start_time = time.time()
                    for _ in range(timed_runs): _ = model(dummy_input)
                    torch.cuda.synchronize()
                    end_time = time.time()
                avg_time_ms = (end_time - start_time) / timed_runs * 1000
                benchmark_results[name]['Speed (ms)'] = avg_time_ms
                print(f"Average Inference Time: {avg_time_ms:.2f} ms/image")
        except Exception as e:
            print(f"Error measuring speed for {name}: {e}")

def generate_summary(benchmark_results):
    print_header("Final Benchmark Summary")
    df = pd.DataFrame.from_dict(benchmark_results, orient='index')
    column_order = ['mAP50-95', 'mAP50', 'Speed (ms)', 'Params (M)', 'FLOPs (G)']
    df = df[[col for col in column_order if col in df.columns]]
    for col in ['mAP50-95', 'mAP50']:
        if col in df.columns: df[col] = df[col].map('{:.4f}'.format)
    for col in ['Params (M)', 'FLOPs (G)', 'Speed (ms)']:
        if col in df.columns: df[col] = df[col].map('{:.2f}'.format)
    print(f"Benchmark conducted on: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
    print(df.to_string())
    df.to_csv("benchmark_summary.csv")
    print("\nSummary table saved to benchmark_summary.csv")
    try:
        df_plot = df.astype(float)
        plt.figure(figsize=(10, 6))
        plt.scatter(df_plot['Speed (ms)'], df_plot['mAP50-95'])
        for i, txt in enumerate(df_plot.index):
            plt.annotate(txt, (df_plot['Speed (ms)'].iloc[i], df_plot['mAP50-95'].iloc[i]), ha='right', va='bottom')
        plt.title('Benchmark: Speed vs. Accuracy')
        plt.xlabel('Inference Time (ms)')
        plt.ylabel('mAP @ .50-.95')
        plt.grid(True)
        plt.savefig("benchmark_plot.png")
        plt.show()
        print("Summary plot saved to benchmark_plot.png")
    except Exception as e:
        print(f"Could not generate plot: {e}")

def main():
    warnings.filterwarnings("ignore", category=UserWarning)
    benchmark_results = OrderedDict()
    yolo_data_yaml_path = prepare_yolo_dataset()
    base_data_path = "/kaggle/input/rt-detr-dinov3-distilled-model/FINAL"
    paths = setup_and_verify(base_data_path)
    if paths:
        evaluate_models(paths, yolo_data_yaml_path, YOLO, benchmark_results)
        calculate_complexity(paths, YOLO, profile, YAMLConfig, benchmark_results)
        measure_inference_speed(paths, YOLO, YAMLConfig, benchmark_results)
    generate_summary(benchmark_results)
    print_header("BENCHMARK PROCESS COMPLETE")

if __name__ == "__main__":
    main()

Writing /kaggle/working/final_benchmark.py


In [5]:
!rm -rf /kaggle/working/RT-DETR
!rm -rf /kaggle/working/taco_yolo
!git clone https://github.com/lyuwenyu/RT-DETR.git
!python /kaggle/working/final_benchmark.py

Cloning into 'RT-DETR'...
remote: Enumerating objects: 1100, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 1100 (delta 8), reused 5 (delta 5), pack-reused 1077 (from 2)
Receiving objects: 100% (1100/1100), 660.70 KiB | 6.06 MiB/s, done.
Resolving deltas: 100% (522/522), done.
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
E0000 00:00:1761137165.496059     128 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761137165.618227     128 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been r